In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
from sympy.codegen import While

driver=Chrome()
driver.get("https://books.toscrape.com")

In [41]:
def get_books_info(links):
    books={}
    for link in links:
        book_info={}
        driver.get(link)
        title=driver.find_element(By.TAG_NAME,"h1").text
        books[title]={}
        table=driver.find_element(By.TAG_NAME,"table")
        trs=table.find_elements(By.TAG_NAME,"tr")
        for tr in trs:
            th=tr.find_element(By.TAG_NAME,"th").text
            td=tr.find_element(By.TAG_NAME,"td").text
            book_info[th]=td

        books[title].update(book_info)
    return books


In [54]:
import time
from selenium.common import NoSuchElementException

books_info_dict={}
current_page_url = "https://books.toscrape.com/index.html"
while current_page_url:
    driver.get(current_page_url)
    main_container=driver.find_element(By.CSS_SELECTOR,"header + *")
    section=main_container.find_element(By.CSS_SELECTOR,"section")
    ol=section.find_element(By.TAG_NAME,"ol")
    books=ol.find_elements(By.TAG_NAME,"li")
    links=[]
    for book in books:
        article=book.find_element(By.TAG_NAME,"article")
        link=article.find_element(By.TAG_NAME,"h3").find_element(By.TAG_NAME,"a").get_attribute("href")
        links.append(link)
    books_info_dict.update(get_books_info(links))
    driver.get(current_page_url)
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, "ul.pager li.next a")
        current_page_url = next_button.get_attribute("href")

    except NoSuchElementException:
        current_page_url = None
print(len(books_info_dict.keys()))


999


In [49]:
# Why not 1000?

In [55]:
import json

print(json.dumps(books_info_dict, indent=2))

{
  "A Light in the Attic": {
    "UPC": "a897fe39b1053632",
    "Product Type": "Books",
    "Price (excl. tax)": "\u00a351.77",
    "Price (incl. tax)": "\u00a351.77",
    "Tax": "\u00a30.00",
    "Availability": "In stock (22 available)",
    "Number of reviews": "0"
  },
  "Tipping the Velvet": {
    "UPC": "90fa61229261140a",
    "Product Type": "Books",
    "Price (excl. tax)": "\u00a353.74",
    "Price (incl. tax)": "\u00a353.74",
    "Tax": "\u00a30.00",
    "Availability": "In stock (20 available)",
    "Number of reviews": "0"
  },
  "Soumission": {
    "UPC": "6957f44c3847a760",
    "Product Type": "Books",
    "Price (excl. tax)": "\u00a350.10",
    "Price (incl. tax)": "\u00a350.10",
    "Tax": "\u00a30.00",
    "Availability": "In stock (20 available)",
    "Number of reviews": "0"
  },
  "Sharp Objects": {
    "UPC": "e00eb4fd7b871a48",
    "Product Type": "Books",
    "Price (excl. tax)": "\u00a347.82",
    "Price (incl. tax)": "\u00a347.82",
    "Tax": "\u00a30.00",
  

In [56]:
import pandas as pd
import re

df=pd.DataFrame.from_dict(books_info_dict,orient="index")
df['StockLevel'] = df['Availability'].str.extract(r'\((\d+) available\)')
df['StockLevel'] = df['StockLevel'].fillna(0)
df['StockLevel'] = df['StockLevel'].astype(int)
statistics = df['StockLevel'].describe()

print(statistics)

count    999.000000
mean       8.579580
std        5.654854
min        1.000000
25%        3.000000
50%        7.000000
75%       14.000000
max       22.000000
Name: StockLevel, dtype: float64
